# Creating and Valuing a CDS Contract vs Markit

In [1]:
import sys
sys.path.append("..")
sys.path.append("..\\..")

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from financepy.finutils.FinDate import FinDate
from financepy.finutils.FinDayCount import FinDayCountTypes
from financepy.finutils.FinFrequency import FinFrequencyTypes
from financepy.finutils.FinMath import ONE_MILLION
from financepy.products.libor.FinLiborDeposit import FinLiborDeposit
from financepy.products.libor.FinLiborSwap import FinLiborSwap
from financepy.market.curves.FinLiborCurve import FinLiborCurve
from financepy.products.credit.FinCDS import FinCDS
from financepy.market.curves.FinCDSCurve import FinCDSCurve

## Creating a CDS Contract

This based on an example from Markit's CDS calculator website https://www.markit.com/markit.jsp?jsppage=pv.jsp

In [4]:
tradeDate = FinDate(21, 11, 2019)
settlementDate = tradeDate.addDays(1)
maturityDate = FinDate(20, 12, 2024)
cdsCoupon = 0.050
notional = ONE_MILLION
longProtection = True
tradeDate = FinDate(2019, 8, 9)

In [5]:
cdsContract = FinCDS(settlementDate, maturityDate, cdsCoupon, notional, longProtection)

In [6]:
cdsContract.print(settlementDate)

STEPINDATE:  FRI 22 NOV 2019
MATURITY:  FRI 20 DEC 2024
NOTIONAL: 1000000
RUNNING COUPON:  500.0 bp
DAYCOUNT:  FinDayCountTypes.ACT_360
FREQUENCY:  FinFrequencyTypes.QUARTERLY
CALENDAR:  FinCalendarTypes.WEEKEND
BUSDAYRULE:  FinDayAdjustTypes.FOLLOWING
DATEGENRULE:  FinDateGenRuleTypes.BACKWARD
ACCRUED DAYS: 63
PAYMENT_DATE      YEAR_FRAC      FLOW
FRI 20 DEC 2019   0.252778     12638.89
FRI 20 MAR 2020   0.252778     12638.89
MON 22 JUN 2020   0.261111     13055.56
MON 21 SEP 2020   0.252778     12638.89
MON 21 DEC 2020   0.252778     12638.89
MON 22 MAR 2021   0.252778     12638.89
MON 21 JUN 2021   0.252778     12638.89
MON 20 SEP 2021   0.252778     12638.89
MON 20 DEC 2021   0.252778     12638.89
MON 21 MAR 2022   0.252778     12638.89
MON 20 JUN 2022   0.252778     12638.89
TUE 20 SEP 2022   0.255556     12777.78
TUE 20 DEC 2022   0.252778     12638.89
MON 20 MAR 2023   0.250000     12500.00
TUE 20 JUN 2023   0.255556     12777.78
WED 20 SEP 2023   0.255556     12777.78
WED 20 DE

## Build Libor Curve

In [7]:
dcType = FinDayCountTypes.ACT_360
depo1 = FinLiborDeposit(settlementDate, "1M", 0.017156, dcType)
depo2 = FinLiborDeposit(settlementDate, "2M", 0.018335, dcType)
depo3 = FinLiborDeposit(settlementDate, "3M", 0.018988, dcType)
depo4 = FinLiborDeposit(settlementDate, "6M", 0.018911, dcType)
depo5 = FinLiborDeposit(settlementDate, "12M", 0.019093, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swap1 = FinLiborSwap(settlementDate,"2Y",0.015630,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"3Y",0.015140,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"4Y",0.015065,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"5Y",0.015140,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"6Y",0.015270,fixedFreq,dcType)
swap6 = FinLiborSwap(settlementDate,"7Y",0.015470,fixedFreq,dcType)
swap7 = FinLiborSwap(settlementDate,"8Y",0.015720,fixedFreq,dcType)
swap8 = FinLiborSwap(settlementDate,"9Y",0.016000,fixedFreq,dcType)
swap9 = FinLiborSwap(settlementDate,"10Y",0.016285,fixedFreq,dcType)
swap10 = FinLiborSwap(settlementDate,"12Y",0.01670,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9,swap10]

liborCurve = FinLiborCurve("USD_LIBOR", settlementDate, depos, [], swaps)

# Build a CDS Curve

In [57]:
cdsFlatSpread = 0.0100

In [58]:
cds1 = FinCDS(settlementDate, "1Y", cdsFlatSpread)
cds2 = FinCDS(settlementDate, "2Y", cdsFlatSpread)
cds3 = FinCDS(settlementDate, "3Y", cdsFlatSpread)
cds4 = FinCDS(settlementDate, "4Y", cdsFlatSpread)
cds5 = FinCDS(settlementDate, "5Y", cdsFlatSpread)
cds6 = FinCDS(settlementDate, "7Y", cdsFlatSpread)
cds7 = FinCDS(settlementDate, "10Y", cdsFlatSpread)
cds8 = FinCDS(settlementDate, "15Y", cdsFlatSpread)

In [59]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [60]:
recoveryRate = 0.40

In [61]:
issuerCurve = FinCDSCurve(settlementDate, cdss, liborCurve, recoveryRate)

# Valuation Results

In [62]:
spd = cdsContract.parSpread(settlementDate, issuerCurve, recoveryRate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  100.00094 bp


In [63]:
fullv, cleanv = cdsContract.value(settlementDate, issuerCurve, recoveryRate)
print("FULL VALUE  %12.2f"% fullv)
print("CLEAN VALUE %12.2f"% cleanv)

FULL VALUE    -198555.99
CLEAN VALUE   -189805.99


MARKIT CALCULATE -198,551

In [64]:
cleanp = cdsContract.cleanPrice(settlementDate, issuerCurve, recoveryRate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   118.980582


MARKIT CALCULATE 118.98%

In [65]:
accruedDays = cdsContract.accruedDays()
print("ACCRUED_DAYS", accruedDays)

ACCRUED_DAYS 63


MARKIT CALCULATE 63

In [66]:
accruedInterest = cdsContract.accruedInterest()
print("ACCRUED_COUPON", accruedInterest)

ACCRUED_COUPON -8750.0


MARKIT CALCULATE 8750

In [67]:
protPV = cdsContract.protectionLegPV(settlementDate, issuerCurve, recoveryRate)
print("PROTECTION_PV", protPV)

PROTECTION_PV 47452.053347467365


In [68]:
premPV = cdsContract.premiumLegPV(settlementDate, issuerCurve, recoveryRate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 246008.04392790023


## Risk Measures

In [71]:
creditDV01 = cdsContract.creditDV01(settlementDate, issuerCurve, recoveryRate)
print("Credit DV01  %12.2f"% creditDV01)

Credit DV01        553.56


MARKIT FOUND 554

In [72]:
interestDV01 = cdsContract.interestDV01(settlementDate, issuerCurve, recoveryRate)
print("Interest DV01  %12.2f"% interestDV01)

Interest DV01         48.79


MARKIT FOUND 49

## Full Analysis 

In [73]:
cdsContract.printFlows(issuerCurve)

PAYMENT_DATE      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
FRI 20 DEC 2019   0.252778     12638.89     0.998668     0.998704     12605.69
FRI 20 MAR 2020   0.252778     12638.89     0.993776     0.994505     12491.19
MON 22 JUN 2020   0.261111     13055.56     0.988911     0.990185     12784.06
MON 21 SEP 2020   0.252778     12638.89     0.984174     0.986021     12264.98
MON 21 DEC 2020   0.252778     12638.89     0.980017     0.981877     12161.85
MON 22 MAR 2021   0.252778     12638.89     0.977129     0.977756     12075.11
MON 21 JUN 2021   0.252778     12638.89     0.974250     0.973651     11988.99
MON 20 SEP 2021   0.252778     12638.89     0.971379     0.969564     11903.48
MON 20 DEC 2021   0.252778     12638.89     0.968349     0.965495     11816.55
MON 21 MAR 2022   0.252778     12638.89     0.964953     0.961443     11725.70
MON 20 JUN 2022   0.252778     12638.89     0.961569     0.957409     11635.54
TUE 20 SEP 2022   0.255556     12777.78     0.958160   